# Feature detection and matching

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

print(cv2.__version__)

%matplotlib inline

# Feature detection
## Harris feature detector
#### Read test data

In [ ]:
dataset_path = './data'
img1_building = cv2.imread(os.path.join(dataset_path, 'montmartre1.jpg'))
height,width,channels = img1_building.shape
img1_building = cv2.resize(img1_building,(round(0.25*width), round(0.25*height)), interpolation = cv2.INTER_LINEAR)
img1_building = cv2.cvtColor(img1_building, cv2.COLOR_BGR2RGB)  # Convert from cv's BRG default color order to RGB
img2_building = cv2.imread(os.path.join(dataset_path, 'montmartre2.jpg'))
height,width,channels = img2_building.shape
img2_building = cv2.resize(img2_building,(round(0.25*width), round(0.25*height)), interpolation = cv2.INTER_LINEAR)
img2_building = cv2.cvtColor(img2_building, cv2.COLOR_BGR2RGB)  # Convert from cv's BRG default color order to RGB

#### Calculate Harris features and show result

In [ ]:
gray = np.float32(cv2.cvtColor(img1_building, cv2.COLOR_RGB2GRAY))
dst = cv2.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
gray[dst>0.01*dst.max()]=255 #[0,0,255]

plt.figure(figsize=(18, 18))
plt.imshow(gray,cmap="gray"); plt.show()

## SIFT features
#### Read test data

In [ ]:
dataset_path = './data'
img1_building = cv2.imread(os.path.join(dataset_path, 'montmartre1.jpg'))
height,width,channels = img1_building.shape
img1_building = cv2.resize(img1_building,(round(0.25*width), round(0.25*height)), interpolation = cv2.INTER_LINEAR)
img1_building = cv2.cvtColor(img1_building, cv2.COLOR_BGR2RGB)  # Convert from cv's BRG default color order to RGB
img2_building = cv2.imread(os.path.join(dataset_path, 'montmartre2.jpg'))
height,width,channels = img2_building.shape
img2_building = cv2.resize(img2_building,(round(0.25*width), round(0.25*height)), interpolation = cv2.INTER_LINEAR)
img2_building = cv2.cvtColor(img2_building, cv2.COLOR_BGR2RGB)  # Convert from cv's BRG default color order to RGB

#### Calculate SIFT features and show result

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
kp1_sift, des1_sift = sift.detectAndCompute(img1_building, None)
img1_kp_sift = cv2.drawKeypoints(img1_building, kp1_sift, img1_building)
kp2_sift, des2_sift = sift.detectAndCompute(img2_building, None)
img2_kp_sift = cv2.drawKeypoints(img2_building, kp2_sift, img2_building)

plt.figure(figsize=(15, 15))
plt.title('SIFT Interest Points')
plt.imshow(img1_kp_sift); plt.show()

## ORB features
#### Read test data

In [ ]:
dataset_path = './data'
img1_building = cv2.imread(os.path.join(dataset_path, 'montmartre1.jpg'))
height,width,channels = img1_building.shape
img1_building = cv2.resize(img1_building,(round(0.25*width), round(0.25*height)), interpolation = cv2.INTER_LINEAR)
img1_building = cv2.cvtColor(img1_building, cv2.COLOR_BGR2RGB)  # Convert from cv's BRG default color order to RGB
img2_building = cv2.imread(os.path.join(dataset_path, 'montmartre2.jpg'))
height,width,channels = img2_building.shape
img2_building = cv2.resize(img2_building,(round(0.25*width), round(0.25*height)), interpolation = cv2.INTER_LINEAR)
img2_building = cv2.cvtColor(img2_building, cv2.COLOR_BGR2RGB)  # Convert from cv's BRG default color order to RGB

#### Calculate ORB features and show result

In [ ]:
orb = cv2.ORB_create()  # OpenCV 3 backward incompatibility: Do not create a detector with `cv2.ORB()`.
kp1_orb, des1_orb = orb.detectAndCompute(img1_building, None)
kp2_orb, des2_orb = orb.detectAndCompute(img2_building, None)
img1_kp_orb = cv2.drawKeypoints(img1_building, kp1_orb, img1_building) # Draw circles.

plt.figure(figsize=(15, 15))
plt.title('ORB Interest Points')
plt.imshow(img1_kp_orb); plt.show()

## Feature matching

#### Brute force matcher

In [ ]:
# BFMatcher with default parameters on SIFT features
bf = cv2.BFMatcher()
matches_bf = bf.knnMatch(des1_sift,des2_sift, k=2) # selecting k=2 best matches

# Apply ratio test to best and second best match
good_sift_bf = []
for m,n in matches_bf:
    if m.distance < 0.75*n.distance:
        good_sift_bf.append(m)

img3 = cv2.drawMatches(img1_building,kp1_sift,img2_building,kp2_sift,good_sift_bf,img1_building,flags=2)

plt.figure(figsize=(15, 15))
plt.imshow(img3)
plt.show()

#### FLANN-based matcher

In [ ]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)
matches_flann = flann.knnMatch(des1_sift, des2_sift, k=2)

# Apply ratio test to best and second best match
matchesMask = [[0, 0] for i in range(len(matches_flann))]
good_sift_flann = []
for i, (m, n) in enumerate(matches_flann):
    if m.distance < 0.55*n.distance:
        matchesMask[i] = [1, 0]
        good_sift_flann.append(m)

draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   matchesMask=matchesMask,
                   flags=0)

img3 = cv2.drawMatchesKnn(img1_building, kp1_sift, img2_building, kp2_sift, matches_flann, None, **draw_params)
plt.figure(figsize=(15, 15))
plt.imshow(img3); plt.show()

#### Brute force matcher on ORB features

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches_orb = bf.match(des1_orb, des2_orb)
matches_orb = sorted(matches_orb, key = lambda x: x.distance) # Sort matches by distance.  Best come first.
    
nmatches = 10
img_matches = cv2.drawMatches(img1_building, kp1_orb, img2_building, kp2_orb, 
                              matches_orb[:nmatches], img2_building, flags=2) # Show top 10 matches
plt.figure(figsize=(16, 16))
plt.title('Brute force matcher on ORB features')
plt.imshow(img_matches); plt.show()

## Model fitting (homography) using matched features

In [ ]:
MIN_MATCH_COUNT = 10
if len(good_sift_flann)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1_sift[m.queryIdx].pt for m in good_sift_flann ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2_sift[m.trainIdx].pt for m in good_sift_flann ]).reshape(-1,1,2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    #M, mask = cv2.findHomography(src_pts, dst_pts, 0,5.0)
    matchesMask = mask.ravel().tolist()

    h,w,d = img1_building.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)

    img2 = cv2.polylines(img2_building,[np.int32(dst)],True,255,3, cv2.LINE_AA)

else:
    print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None

draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

img3 = cv2.drawMatches(img1_building,kp1_sift,img2_building,kp2_sift,good_sift_flann,None,**draw_params)
plt.figure(figsize=(18, 8))
plt.imshow(img3)
plt.show()